In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
BASE_URL = "http://localhost:8000"

In [10]:
requests.get # for get

<function requests.api.get(url, params=None, **kwargs)>

In [11]:
requests.post # for post

<function requests.api.post(url, data=None, json=None, **kwargs)>

In [12]:
requests.delete # for delete

<function requests.api.delete(url, **kwargs)>

In [13]:
requests.put # for update

<function requests.api.put(url, data=None, **kwargs)>

In [19]:
response = requests.get(f"{BASE_URL}")
response.json()

{'message': 'alive'}

In [34]:
response = requests.get(f"{BASE_URL}/v1/session")
response.json()

{'response': ['session1', 'session2']}

In [35]:
response = requests.post(f"{BASE_URL}/v1/session")
response.json()

{'response': 'session created successfully'}

# User

## Register

In [10]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/register", headers=headers, json=payload)
response.json()

{'response': 'success'}

In [11]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/register", headers=headers, json=payload)
response.json()

{'response': 'Constraint Error: Duplicate key "username: bank" violates unique constraint.'}

## Login

In [12]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.json()

{'user_id': '5199c1ff-aef5-4149-a77d-ca197ad92330', 'username': 'bank'}

In [13]:
user_id = response.json()['user_id']
user_id

'5199c1ff-aef5-4149-a77d-ca197ad92330'

In [14]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bonk",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.json()

{'response': 'username: bonk not found'}

In [15]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssward",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.json()

{'response': 'incorrect password'}

# Session

## Create

In [16]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/create", headers=headers, json=payload)
response.json()

{'response': 'success'}

## list

In [17]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': [{'session_id': '57e9aebe-35dc-45a5-bb2a-5e748aa18d0f',
   'user_id': '5199c1ff-aef5-4149-a77d-ca197ad92330',
   'tone_of_voice': None,
   'outline': None,
   'step': 'not start',
   'step_remark': None,
   'knowledge': None,
   'enrich': None,
   'publish': None,
   'created_at': '2025-05-04T18:39:11.214452',
   'updated_at': '2025-05-04T18:39:11.214452'}]}

In [18]:
session_id = response.json()['response'][0]['session_id']
session_id

'57e9aebe-35dc-45a5-bb2a-5e748aa18d0f'

In [19]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": "fake",
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': []}

## research: update_tone_outline

In [20]:
headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "tone_of_voice": "tone",
    "outline": "outline123"
}

response = requests.put(f"{BASE_URL}/v1/session/update-tone-of-voice-and-outline", headers=headers, json=payload)
response.json()

{'response': 'success'}

In [21]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': [{'session_id': '57e9aebe-35dc-45a5-bb2a-5e748aa18d0f',
   'user_id': '5199c1ff-aef5-4149-a77d-ca197ad92330',
   'tone_of_voice': 'tone',
   'outline': 'outline123',
   'step': 'not start',
   'step_remark': None,
   'knowledge': None,
   'enrich': None,
   'publish': None,
   'created_at': '2025-05-04T18:39:11.214452',
   'updated_at': '2025-05-04T18:39:17.482980'}]}

# Test

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables
from package.database.user import UserDB, UserRegister, UserLogin
from package.database.session import SessionDB, Knowledge, KnowledgeSection, KnowledgeQuestion, SessionKnowledge, SessionStep
from package.database.session import SessionEnrich, EnrichSection, EnrichQuestion, Enrich, SessionPublish

In [23]:
userDB = UserDB(db_name=os.getenv("DB_NAME"))
sessionDB = SessionDB(db_name=os.getenv("DB_NAME"))

In [24]:
userDB.read_all()

,user_id,username,password,created_at,updated_at
0,5199c1ff-aef5-4149-a77d-ca197ad92330,bank,P@ssw0rd,2025-05-04 18:39:07.096797,2025-05-04 18:39:07.096797


In [25]:
sessionDB.read_all()

,session_id,user_id,tone_of_voice,outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,57e9aebe-35dc-45a5-bb2a-5e748aa18d0f,5199c1ff-aef5-4149-a77d-ca197ad92330,tone,outline123,not start,None,None,None,None,2025-05-04 18:39:11.214452,2025-05-04 18:39:17.482980


In [43]:
userDB.drop_table()

,Success


In [44]:
sessionDB.drop_table()

,Success


In [26]:
kl = Knowledge(sections=[
    KnowledgeSection(
        section="Test S1",
        questions=[
            KnowledgeQuestion(
                question="Test Q1",
                retrieved_ids=["a", "b", "c"]
            ),
            KnowledgeQuestion(
                question="Test Q2",
                retrieved_ids=["a", "b", "c"]
            )
        ]
    ),
    KnowledgeSection(
        section="Test S2",
        questions=[
            KnowledgeQuestion(
                question="Test Q1",
                retrieved_ids=["a", "b", "c"]
            ),
            KnowledgeQuestion(
                question="Test Q2",
                retrieved_ids=["a", "b", "c"]
            )
        ]
    ),
])
skl = SessionKnowledge(session_id=session_id, step=SessionStep.ENRICH, knowledge=kl)

In [27]:
sessionDB.update_knowledge(skl)

,Count
0,1


In [28]:
sessionDB.read_all()

,session_id,user_id,tone_of_voice,outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,57e9aebe-35dc-45a5-bb2a-5e748aa18d0f,5199c1ff-aef5-4149-a77d-ca197ad92330,tone,outline123,enrich,None,"{""sections"":[{""section"":""Test S1"",""questions"":...",None,None,2025-05-04 18:39:11.214452,2025-05-04 18:39:29.970969


In [29]:
import json
s_dict = sessionDB.read_all().to_dict(orient="records")[0]
SessionKnowledge(session_id=s_dict.get("session_id"), knowledge=json.loads(s_dict.get("knowledge")))

SessionKnowledge(created_at='2025-05-04 18:39:41.420946', step='not start', step_remark=None, session_id='57e9aebe-35dc-45a5-bb2a-5e748aa18d0f', knowledge=Knowledge(sections=[KnowledgeSection(section='Test S1', questions=[KnowledgeQuestion(question='Test Q1', retrieved_ids=['a', 'b', 'c']), KnowledgeQuestion(question='Test Q2', retrieved_ids=['a', 'b', 'c'])]), KnowledgeSection(section='Test S2', questions=[KnowledgeQuestion(question='Test Q1', retrieved_ids=['a', 'b', 'c']), KnowledgeQuestion(question='Test Q2', retrieved_ids=['a', 'b', 'c'])])]))

In [30]:
en = Enrich(
    sections=[
        EnrichSection(
            section="Test S1",
            questions=[
                EnrichQuestion(question="Test Q1", answer="Answer Q1"),
                EnrichQuestion(question="Test Q2", answer="Answer Q2"),
            ]
        ),
        EnrichSection(
            section="Test S2",
            questions=[
                EnrichQuestion(question="Test Q1", answer="Answer Q1"),
                EnrichQuestion(question="Test Q2", answer="Answer Q2"),
            ]
        ),
    ]
)
sen = SessionEnrich(session_id=session_id, enrich=en, step=SessionStep.REVISE)
sen

SessionEnrich(created_at='2025-05-04 18:39:43.302944', step='revise', step_remark=None, session_id='57e9aebe-35dc-45a5-bb2a-5e748aa18d0f', enrich=Enrich(sections=[EnrichSection(section='Test S1', questions=[EnrichQuestion(question='Test Q1', answer='Answer Q1'), EnrichQuestion(question='Test Q2', answer='Answer Q2')]), EnrichSection(section='Test S2', questions=[EnrichQuestion(question='Test Q1', answer='Answer Q1'), EnrichQuestion(question='Test Q2', answer='Answer Q2')])]))

In [31]:
sen.model_dump()

/home/ec2-user/SageMaker/broai-research-rocket/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `datetime` - serialized value may not be as expected [input_value='2025-05-04 18:39:43.302944', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [input_value='revise', input_type=str])
  return self.__pydantic_serializer__.to_python(


{'created_at': '2025-05-04 18:39:43.302944',
 'step': 'revise',
 'step_remark': None,
 'session_id': '57e9aebe-35dc-45a5-bb2a-5e748aa18d0f',
 'enrich': {'sections': [{'section': 'Test S1',
    'questions': [{'question': 'Test Q1', 'answer': 'Answer Q1'},
     {'question': 'Test Q2', 'answer': 'Answer Q2'}]},
   {'section': 'Test S2',
    'questions': [{'question': 'Test Q1', 'answer': 'Answer Q1'},
     {'question': 'Test Q2', 'answer': 'Answer Q2'}]}]}}

In [32]:
sessionDB.update_enrich(sen)

,Count
0,1


In [33]:
sessionDB.read_all()

,session_id,user_id,tone_of_voice,outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,57e9aebe-35dc-45a5-bb2a-5e748aa18d0f,5199c1ff-aef5-4149-a77d-ca197ad92330,tone,outline123,revise,None,"{""sections"":[{""section"":""Test S1"",""questions"":...","{""sections"":[{""section"":""Test S1"",""questions"":...",None,2025-05-04 18:39:11.214452,2025-05-04 18:39:43.302944


In [36]:
sessionDB.update_publish(SessionPublish(session_id=session_id, step=SessionStep.DONE, publish="Test Publish"))

,Count
0,1


In [37]:
sessionDB.read_all()

,session_id,user_id,tone_of_voice,outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,57e9aebe-35dc-45a5-bb2a-5e748aa18d0f,5199c1ff-aef5-4149-a77d-ca197ad92330,tone,outline123,done,None,"{""sections"":[{""section"":""Test S1"",""questions"":...","{""sections"":[{""section"":""Test S1"",""questions"":...",Test Publish,2025-05-04 18:39:11.214452,2025-05-04 18:41:35.432928
